In [1]:
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder

##### Input variables

In [2]:
unfiltered_v1 = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\variable_p39.csv")
unfiltered_v2 = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\variable_p40.csv")
unfiltered_v3 = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\variable_p41.csv")

##### Construction

In [3]:
# 先不排除任何資料
v1 = unfiltered_v1
v2 = unfiltered_v2
v3 = unfiltered_v3

In [4]:
#1.
v1['accrual_wr'] = (v1['oancfy']-v1['ibq'])/v1['atq']

#2.
v1['aftret_eq_wr'] = v1['ibcomq']/v1['ceqq']

#3.
v1['aftret_equity_wr'] = v1['ibq']/v1['seqq']

#4.
df = v1.sort_values(by=['gvkey', 'datadate'])  # 先按公司和日期排序
# 計算 lag 變數
df['l_icaptq'] = df.groupby('gvkey')['icaptq'].shift(1)  # 上一期的 Invested Capital
df['l_txditcq'] = df.groupby('gvkey')['txditcq'].shift(1)  # 上一期的 Deferred Taxes
df['l_mibq'] = df.groupby('gvkey')['mibq'].shift(1)  # 上一期的 Minority Interest Balance
# 計算分子
df['numerator'] = df['ibq'] + df['xintq'] + df['miiq']
# 計算分母
df['denominator'] = df['l_icaptq'] + df['l_txditcq'] - df['l_mibq']
# 計算 After-tax Return on Invested Capital，分母為正時才計算
df['after_tax_roic'] = df['numerator'] / df['denominator']
df.loc[df['denominator'] <= 0, 'after_tax_roic'] = None  # 分母為負或零時設為 None

#5.
v1['at_turn_wr'] = v1['saleq']/v1['atq']

#6.
v1['capital_ratio_wr'] = v1['dlttq']/(v1['dlttq']+v1['ceqq']+v1['pstkq'])

#7.
# 計算 lag 變數（上一期的值）
df['l_invtq'] = df.groupby('gvkey')['invtq'].shift(1)
df['l_rectq'] = df.groupby('gvkey')['rectq'].shift(1)
df['l_apq'] = df.groupby('gvkey')['apq'].shift(1)

# 計算 Days Inventory Outstanding (DIO)
df['DIO'] = (df['invtq'] + df['l_invtq']) / 2 / (df['cogsq'] / 365)

# 計算 Days Sales Outstanding (DSO)
df['DSO'] = (df['rectq'] + df['l_rectq']) / 2 / (df['saleq'] / 365)

# 計算 Days Payable Outstanding (DPO)
df['DPO'] = (df['apq'] + df['l_apq']) / 2 / (df['cogsq'] / 365)

# 計算 Cash Conversion Cycle (CCC)
df['cash_conversion_wr'] = df['DIO'] + df['DSO'] - df['DPO']

#8.
v1['cash_debt_wr'] = (v1['ibq']+v1['dpq'])/v1['ltq']

#9.
v1['cash_lt_wr'] = v1['cheq']/v1['ltq']

#10.
v1['cash_ratio_wr'] = v1['cheq']/v1['lctq']

#11.
v1['cfm_wr'] = (v1['ibq']+v1['dpq'])/v1['saleq']

#12.
v1['curr_debt_wr'] = v1['lctq']/v1['ltq']

#13.
v1['curr_ratio_wr'] = v1['actq']/v1['lctq']

#14.
v1['de_ratio_wr'] = v1['ltq']/(v1['ceqq']+v1['pstkq'])

#15.
v1['debt_assets_wr'] = v1['ltq']/v1['atq']

#16.
v1['debt_at_wr'] = (v1['dlttq']+v1['dlcq'])/v1['atq']

#17.
v1['debt_book_value'] = v1['dlttq']+v1['dlcq']+v1['pstkq']

#18.
v1['atq_wr'] = v1['atq']

#19.
v1['ceqq_wr'] = v1['ceqq']

#20. 
v1['saleq_wr'] = v1['saleq'] 

#21. 
v1['total_debt_wr'] = v1['dlttq']+v1['dlcq']

In [5]:
# 使用 'gvkey' 和 'datadate' 作為鍵進行合併
v1 = v1.merge(df[['gvkey', 'datadate', 'after_tax_roic','cash_conversion_wr']], on=['gvkey', 'datadate'], how='left')
v1

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,de_ratio_wr,debt_assets_wr,debt_at_wr,debt_book_value,atq_wr,ceqq_wr,saleq_wr,total_debt_wr,after_tax_roic,cash_conversion_wr
0,1001,1983-03-31,1983,1.0,INDL,C,D,STD,AMFD.,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,4.921,NaN,NaN,NaN
1,1001,1983-06-30,1983,2.0,INDL,C,D,STD,AMFD.,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,5.859,NaN,NaN,NaN
2,1001,1983-09-30,1983,3.0,INDL,C,D,STD,AMFD.,USD,...,3.815494,0.792337,0.621190,5.869,9.448,1.962,6.449,5.869,NaN,NaN
3,1001,1983-12-31,1983,4.0,INDL,C,D,STD,AMFD.,USD,...,0.799821,0.444389,0.345455,4.864,14.080,7.823,8.166,4.864,0.072727,-81.922742
4,1001,1984-03-31,1984,1.0,INDL,C,D,STD,AMFD.,USD,...,0.749629,0.428450,0.326590,4.617,14.137,8.080,6.434,4.617,0.031972,-78.042938
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643199,351491,2019-12-31,2019,4.0,INDL,C,D,STD,IVCGF,USD,...,4.920851,0.829103,0.323640,5776.000,17847.000,3007.000,13408.000,5776.000,NaN,NaN
1643200,351590,2019-03-31,2019,1.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643201,351590,2019-06-30,2019,2.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643202,351590,2019-09-30,2019,3.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [6]:
#1. 
v2['quick_ratio_wr'] = (v2['actq']-v2['invtq'])/v2['lctq']

#2. 
# 替換 xrd 的缺失值為 0
v2['xrdq'] = v2['xrdq'].fillna(0)
# 計算 R&D/Sales 比率並確保最小值為 0
v2['rd_sale_wr'] = (v2['xrdq'] / v2['saleq']).clip(lower=0)

#3.
v2['rect_act_wr'] = v2['rectq']/v2['actq']

#4.
df = v2.sort_values(by=['gvkey', 'datadate'])
df['l_rectq'] = df.groupby('gvkey')['rectq'].shift(1)  
df['avg_rectq'] = (df['rectq'] + df['l_rectq']) / 2
df['rect_turn_wr'] = (df['saleq']) / df['avg_rectq']
df.loc[df['avg_rectq'] == 0, 'rect_turn_wr'] = None  

#5.
v2['roa_wr'] = v2['oibdpq']/v2['atq']

#6.
v2['roce_wr'] = v2['oiadpq']/(v2['dlttq']+v2['pstkq']+v2['dlcq']+v2['ceqq'])

#7.
v2['roe_wr'] = v2['ibq']/(v2['atq']-v2['ltq'])

#8. 
v2['short_debt_wr'] = v2['dlcq']/(v2['dlttq']+v2['dlcq'])

#9. 
v2['totdebt_invcap_wr'] = (v2['dlttq']+v2['dlcq'])/v2['icaptq']
v2.loc[v2['icaptq'] <= 0, 'totdebt_invcap_wr'] = None

In [7]:
v2 = v2.merge(df[['gvkey', 'datadate', 'rect_turn_wr']], on=['gvkey', 'datadate'], how='left')
v2

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,sic,quick_ratio_wr,rd_sale_wr,rect_act_wr,roa_wr,roce_wr,roe_wr,short_debt_wr,totdebt_invcap_wr,rect_turn_wr
0,1001,1983-03-31,1983,1.0,INDL,C,D,STD,AMFD.,USD,...,5812,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1001,1983-06-30,1983,2.0,INDL,C,D,STD,AMFD.,USD,...,5812,NaN,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1001,1983-09-30,1983,3.0,INDL,C,D,STD,AMFD.,USD,...,5812,0.515319,0.000000,0.009057,0.085203,0.083131,0.185525,0.247231,0.919906,NaN
3,1001,1983-12-31,1983,4.0,INDL,C,D,STD,AMFD.,USD,...,5812,2.343962,0.000000,0.017058,0.075639,0.046977,0.031829,0.106908,0.399770,164.969697
4,1001,1984-03-31,1984,1.0,INDL,C,D,STD,AMFD.,USD,...,5812,1.980635,0.000000,0.030047,0.043078,0.027644,0.031807,0.090752,0.376038,63.389163
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643199,351491,2019-12-31,2019,4.0,INDL,C,D,STD,IVCGF,USD,...,3711,0.687718,0.038857,0.701296,0.087522,0.074007,0.030820,0.962777,1.769066,NaN
1643200,351590,2019-03-31,2019,1.0,INDL,C,D,STD,DTRUY,USD,...,3537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643201,351590,2019-06-30,2019,2.0,INDL,C,D,STD,DTRUY,USD,...,3537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643202,351590,2019-09-30,2019,3.0,INDL,C,D,STD,DTRUY,USD,...,3537,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
#1.
v3['debt_capital_wr'] = (v3['apq']+v3['dlcq']+v3['dlttq'])/(v3['apq']+v3['dlcq']+v3['dlttq']+v3['ceqq']+v3['pstkq'])

#2.
v3['debt_ebitda_wr'] = (v3['dlttq']+v3['dlcq'])/v3['oibdpq']

#3.
v3['debt_invcap_wr'] = v3['dlttq']/v3['icaptq']

#4. 
v3['dltt_be_wr'] = v3['dlttq']/(v3['atq']-v3['ltq'])

#5. 
v3['dpr_wr'] = v3['dvy']/v3['ibadjq']

#6.
v3['efftax_wr'] = v3['txtq']/v3['piq']

#7.
v3['equity_invcap_wr'] = v3['ceqq']/v3['icaptq']

#8.
v3['fcf_ocf_wr'] = (v3['oancfy']-v3['capxy'])/v3['oancfy']

#9.
v3['gpm_wr'] = (v3['saleq']-v3['cogsq'])/v3['saleq']

#10.
v3['gprof_wr'] = (v3['saleq']-v3['cogsq'])/v3['atq']

#11.
v3['int_debt_wr'] = v3['xintq']/v3['dlttq']

#12.
v3['int_totdebt_wr'] = v3['xintq']/(v3['dlttq']+v3['dlcq'])

#13.
v3['intcov_ratio_wr'] = v3['oiadpq']/v3['xintq']

#14.
v3['intcov_wr'] = (v3['xintq']+v3['ibq'])/v3['xintq']

#15.
v3['invt_act_wr'] = v3['invtq']/v3['actq']

#16.
df = v3.sort_values(by=['gvkey', 'datadate'])  # 先按公司和日期排序
# 計算 lag 變數
df['l_invtq'] = df.groupby('gvkey')['invtq'].shift(1) 
# 計算分子
df['numerator'] = df['cogsq'] 
# 計算分母
df['denominator'] = (df['invtq'] + df['l_invtq'])/2
# 分母為正時才計算
df['invturn_wr'] = df['numerator'] / df['denominator']
df.loc[df['denominator'] <= 0, 'invturn_wr'] = None  # 分母為負或零時設為 None

#17.
v3['lt_debt_wr'] = v3['dlttq']/v3['ltq']

#18.
v3['lt_ppent_wr'] = v3['ltq']/v3['ppentq']

#19.
v3['npm_wr'] = v3['ibq']/v3['saleq']

#20.
v3['ocf_lct_wr'] = v3['oancfy']/v3['lctq']

#21.
v3['opmad_wr'] = v3['oiadpq']/v3['saleq']

#22.
v3['opmbd_wr'] = v3['oibdpq']/v3['saleq']

#23.
df = df.sort_values(by=['gvkey', 'datadate'])

# 計算存貨的變動值 (D_INVT)
df['d_invtq'] = df['invtq'] - df['l_invtq']  # 當期存貨變動
# 計算上期應付賬款 (L_AP)
df['l_apq'] = df.groupby('gvkey')['apq'].shift(1)  # 上一期的應付賬款
# 計算應付賬款的平均值
df['avg_apq'] = (df['apq'] + df['l_apq']) / 2
# 計算 Payables Turnover，當 avg_ap > 0 才計算
df['pay_turn_wr'] = (df['cogsq'] + df['d_invtq']) / df['avg_apq']
df.loc[df['avg_apq'] <= 0, 'pay_turn_wr'] = None  # 當 avg_ap 小於或等於 0 時設為 None

#24.
v3['pretret_earnat_wr'] = v3['oibdpq']/(v3['ppentq']+v3['actq'])

#25.
v3['pretret_noa_wr'] = v3['oiadpq']/(v3['ppentq']+v3['actq']-v3['lctq'])

#26.
v3['profit_lct_wr'] = v3['oibdpq']/v3['lctq']

#27.
v3['ptpm_wr'] = (v3['oiadpq']-v3['xintq']+v3['spiq']+v3['nopiq'])/v3['saleq']

In [9]:
v3 = v3.merge(df[['gvkey', 'datadate', 'invturn_wr','pay_turn_wr']], on=['gvkey', 'datadate'], how='left')
v3

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,npm_wr,ocf_lct_wr,opmad_wr,opmbd_wr,pretret_earnat_wr,pretret_noa_wr,profit_lct_wr,ptpm_wr,invturn_wr,pay_turn_wr
0,1001,1983-03-31,1983,1.0,INDL,C,D,STD,AMFD.,USD,...,0.014225,NaN,0.022353,0.057712,NaN,NaN,NaN,0.007519,NaN,NaN
1,1001,1983-06-30,1983,2.0,INDL,C,D,STD,AMFD.,USD,...,0.077146,NaN,0.108892,0.148490,NaN,NaN,NaN,0.106503,NaN,NaN
2,1001,1983-09-30,1983,3.0,INDL,C,D,STD,AMFD.,USD,...,0.056443,NaN,0.100946,0.124826,0.095583,0.121591,0.262386,0.087300,NaN,NaN
3,1001,1983-12-31,1983,4.0,INDL,C,D,STD,AMFD.,USD,...,0.030492,NaN,0.072986,0.130419,0.079817,0.052143,0.556717,0.042861,6.174475,2.584000
4,1001,1984-03-31,1984,1.0,INDL,C,D,STD,AMFD.,USD,...,0.039944,NaN,0.054554,0.094653,0.045492,0.030448,0.327595,0.061859,4.694611,2.291066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1643199,351491,2019-12-31,2019,4.0,INDL,C,D,STD,IVCGF,USD,...,0.007011,0.063235,0.048479,0.116498,0.101892,0.237053,0.124086,0.014618,NaN,NaN
1643200,351590,2019-03-31,2019,1.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643201,351590,2019-06-30,2019,2.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1643202,351590,2019-09-30,2019,3.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
v1 = v1.merge(v2, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'curcdq', 'datacqtr', 'datafqtr','costat', 'naics', 'sic'], how='left')
v1 = v1.merge(v3, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'curcdq', 'datacqtr', 'datafqtr','costat', 'naics', 'sic'], how='left')
v1

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,npm_wr,ocf_lct_wr,opmad_wr,opmbd_wr,pretret_earnat_wr,pretret_noa_wr,profit_lct_wr,ptpm_wr,invturn_wr,pay_turn_wr
0,1001,1983-03-31,1983,1.0,INDL,C,D,STD,AMFD.,USD,...,0.014225,NaN,0.022353,0.057712,NaN,NaN,NaN,0.007519,NaN,NaN
1,1001,1983-06-30,1983,2.0,INDL,C,D,STD,AMFD.,USD,...,0.077146,NaN,0.108892,0.148490,NaN,NaN,NaN,0.106503,NaN,NaN
2,1001,1983-09-30,1983,3.0,INDL,C,D,STD,AMFD.,USD,...,0.056443,NaN,0.100946,0.124826,0.095583,0.121591,0.262386,0.087300,NaN,NaN
3,1001,1983-12-31,1983,4.0,INDL,C,D,STD,AMFD.,USD,...,0.030492,NaN,0.072986,0.130419,0.079817,0.052143,0.556717,0.042861,6.174475,2.584000
4,1001,1984-03-31,1984,1.0,INDL,C,D,STD,AMFD.,USD,...,0.039944,NaN,0.054554,0.094653,0.045492,0.030448,0.327595,0.061859,4.694611,2.291066
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659651,351491,2019-12-31,2019,4.0,INDL,C,D,STD,IVCGF,USD,...,0.007011,0.063235,0.048479,0.116498,0.101892,0.237053,0.124086,0.014618,NaN,NaN
1659652,351590,2019-03-31,2019,1.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659653,351590,2019-06-30,2019,2.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1659654,351590,2019-09-30,2019,3.0,INDL,C,D,STD,DTRUY,USD,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


##### 篩選欄位

In [11]:
# 篩選出以 'wr' 結尾的欄位
wr_columns = [col for col in v1.columns if col.endswith('wr')]

# 指定要保留的其他欄位
other_columns = ['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic']

# 合併兩個欄位列表
columns_to_keep = other_columns + wr_columns

# 篩選 DataFrame 中指定的欄位
df = v1[columns_to_keep]
df.head()

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,npm_wr,ocf_lct_wr,opmad_wr,opmbd_wr,pretret_earnat_wr,pretret_noa_wr,profit_lct_wr,ptpm_wr,invturn_wr,pay_turn_wr
0,1001,1983-03-31,1983,1.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,0.014225,NaN,0.022353,0.057712,NaN,NaN,NaN,0.007519,NaN,NaN
1,1001,1983-06-30,1983,2.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,0.077146,NaN,0.108892,0.148490,NaN,NaN,NaN,0.106503,NaN,NaN
2,1001,1983-09-30,1983,3.0,AMFD.,722.0,5812,NaN,0.185525,0.185525,...,0.056443,NaN,0.100946,0.124826,0.095583,0.121591,0.262386,0.087300,NaN,NaN
3,1001,1983-12-31,1983,4.0,AMFD.,722.0,5812,NaN,0.031829,0.031829,...,0.030492,NaN,0.072986,0.130419,0.079817,0.052143,0.556717,0.042861,6.174475,2.584000
4,1001,1984-03-31,1984,1.0,AMFD.,722.0,5812,NaN,0.031807,0.031807,...,0.039944,NaN,0.054554,0.094653,0.045492,0.030448,0.327595,0.061859,4.694611,2.291066


-----

##### Target variables

In [6]:
t1 = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\target variables.csv")

In [7]:
t1.head()

,gvkey,datadate,fyearq,fqtr,indfmt,consol,popsrc,datafmt,tic,curcdq,...,atq,ceqq,cheq,dlcq,dlttq,saleq,costat,mkvaltq,naics,sic
0,1001,1983-03-31,1983,1.0,INDL,C,D,STD,AMFD.,USD,...,NaN,NaN,NaN,NaN,NaN,4.921,I,NaN,722.0,5812
1,1001,1983-06-30,1983,2.0,INDL,C,D,STD,AMFD.,USD,...,NaN,NaN,NaN,NaN,NaN,5.859,I,NaN,722.0,5812
2,1001,1983-09-30,1983,3.0,INDL,C,D,STD,AMFD.,USD,...,9.448,1.962,1.410,1.451,4.418,6.449,I,NaN,722.0,5812
3,1001,1983-12-31,1983,4.0,INDL,C,D,STD,AMFD.,USD,...,14.080,7.823,4.280,0.520,4.344,8.166,I,NaN,722.0,5812
4,1001,1984-03-31,1984,1.0,INDL,C,D,STD,AMFD.,USD,...,14.137,8.080,3.318,0.419,4.198,6.434,I,NaN,722.0,5812


In [8]:
t1['m2b'] = t1['mkvaltq']/t1['ceqq']
t1['v2a'] = (t1['mkvaltq']+t1['dlttq']+t1['dlcq']-t1['cheq'])/t1['atq']
t1['v2s'] = (t1['mkvaltq']+t1['dlttq']+t1['dlcq'])/t1['saleq']
# t1['month'] = pd.to_datetime(t1['datadate']).dt.to_period('M')
t1['lnm2b'] = np.log10(t1['m2b'])
t1['lnv2a'] = np.log10(t1['v2a'])
t1['lnv2s'] = np.log10(t1['v2s'])

c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)
c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\pandas\core\arraylike.py:399: RuntimeWarning: divide by zero encountered in log10
  result = geta

In [17]:
# Ensure 'datadate' is in datetime format
t1['datadate'] = pd.to_datetime(t1['datadate'], errors='coerce')

# Extract month-year period
t1['month'] = t1['datadate'].dt.to_period('M')

# Calculate missing and non-missing value counts for each column grouped by month-year
missing_values_count = t1.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.isnull().sum())
non_missing_values_count = t1.groupby('month')[['mkvaltq', 'ceqq', 'dlttq', 'dlcq', 'cheq', 'atq', 'saleq', 'm2b', 'v2a', 'v2s']].apply(lambda x: x.notnull().sum())

# Display both counts side by side
combined_counts = pd.concat([missing_values_count, non_missing_values_count], axis=1, keys=['Missing', 'Not Missing'])

# Print the combined DataFrame
combined_counts

Missing                                                           \
        mkvaltq  ceqq dlttq  dlcq  cheq   atq saleq    m2b    v2a    v2s   
month                                                                      
1980-01     377    11    12    43     9     7     5    377    377    377   
1980-02     255    15    13    34    16    10     4    255    255    255   
1980-03    2248   144   119   325   132    84    45   2248   2248   2248   
1980-04     376    18    16    56    16    13     7    376    376    376   
1980-05     254    19    17    41    21    14     5    254    254    254   
1980-06    2250   145   124   337   151    93    42   2250   2250   2250   
1980-07     374    20    17    58    19    13     8    374    374    374   
1980-08     251    15    15    39    19    13     5    251    251    251   
1980-09    2246   157   137   351   161   103    48   2246   2246   2246   
1980-10     382    29    30    69    30    23     9    382    382    382   
1980-11     253    19    22    43    23    17     1    253    253    253   
1980-12    2262    88    99   215   103    81    32   2262   2262   2262   
1981-01     395    40    41    78    39    34    12    395    395    395   
1981-02     254    33    34    59    32    30     6    254    254    254   
1981-03    3700  1397  1392  1616  1406  1344   118   3700   3700   3700   
1981-04     525   150   150   193   152   143    15    525    525    525   
1981-05     346   110   112   140   111   107    11    346    346    346   
1981-06    3942  1606  1603  1824  1622  1558   121   3942   3942   3942   
1981-07     645   251   249   291   251   242    30    645    645    645   
1981-08     480   216   218   245   219   214    25    480    480    480   
1981-09    4328  1940  1934  2158  1954  1893   174   4328   4328   4328   
1981-10     741   320   322   364   323   315    36    741    741    741   
1981-11     617   305   305   334   305   302    31    617    617    617   
1981-12    4534   876   895  1038   898   871   131   4534   4534   4534   
1982-01     877   316   322   365   323   315    43    877    877    877   
1982-02     695   298   298   330   299   293    40    695    695    695   
1982-03    5076  1145  1157  1519  1167  1080   400   5076   5076   5076   
1982-04     914   248   250   306   255   245    37    914    914    914   
1982-05     713   215   217   258   217   211    32    713    713    713   
1982-06    5124   855   870  1235   882   787   352   5124   5124   5124   
1982-07     933   200   203   263   201   194    33    933    933    933   
1982-08     730   131   140   187   136   129    33    730    730    730   
1982-09    5155   706   719  1102   736   635   363   5155   5155   5155   
1982-10     959   112   115   188   112   103    36    959    959    959   
1982-11     747    80    90   147    81    75    31    747    747    747   
1982-12    5116   426   453   650   447   407   251   5116   5116   5116   
1983-01     972   109   115   182   108   101    48    972    972    972   
1983-02     761   104   110   171   102    98    31    761    761    761   
1983-03    5364   782   801  1187   810   712   371   5364   5364   5364   
1983-04     973   113   118   185   113   108    32    973    973    973   
1983-05     774    95    99   157    94    88    30    774    774    774   
1983-06    5366   736   758  1144   770   669   347   5366   5366   5366   
1983-07     986   102   105   180   101    93    34    986    986    986   
1983-08     787    96   100   159    92    88    32    787    787    787   
1983-09    5407   726   748  1140   756   661   387   5407   5407   5407   
1983-10     998    90    96   168    91    84    30    998    998    998   
1983-11     785    81    83   139    75    72    26    785    785    785   
1983-12    5347   407   440   615   437   391   283   5347   5347   5347   
1984-01     996    86    96   154    87    82    36    996    996    996   
1984-02     783    78    83   144    76    73    32    7

In [15]:
selected_t1 = t1[['m2b','v2a','v2s','lnm2b','lnv2a','lnv2s','gvkey', 'datadate', 'fyearq', 'fqtr', 'indfmt', 'consol', 'popsrc',
       'datafmt', 'tic', 'curcdq', 'datacqtr', 'datafqtr','costat', 'naics', 'sic']]
df = df.merge(selected_t1, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic'], how='left')
df

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,lnv2a,lnv2s,indfmt,consol,popsrc,datafmt,curcdq,datacqtr,datafqtr,costat
0,1001,1983-03-31,1983,1.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,NaN,NaN,INDL,C,D,STD,USD,1983Q1,1983Q1,I
1,1001,1983-06-30,1983,2.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,NaN,NaN,INDL,C,D,STD,USD,1983Q2,1983Q2,I
2,1001,1983-09-30,1983,3.0,AMFD.,722.0,5812,NaN,0.185525,0.185525,...,NaN,NaN,INDL,C,D,STD,USD,1983Q3,1983Q3,I
3,1001,1983-12-31,1983,4.0,AMFD.,722.0,5812,NaN,0.031829,0.031829,...,NaN,NaN,INDL,C,D,STD,USD,1983Q4,1983Q4,I
4,1001,1984-03-31,1984,1.0,AMFD.,722.0,5812,NaN,0.031807,0.031807,...,NaN,NaN,INDL,C,D,STD,USD,1984Q1,1984Q1,I
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659651,351491,2019-12-31,2019,4.0,IVCGF,336120.0,3711,0.039334,0.031260,0.031260,...,NaN,NaN,INDL,C,D,STD,USD,2019Q4,2019Q4,A
1659652,351590,2019-03-31,2019,1.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,NaN,INDL,C,D,STD,USD,2019Q1,2019Q1,A
1659653,351590,2019-06-30,2019,2.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,NaN,INDL,C,D,STD,USD,2019Q2,2019Q2,A
1659654,351590,2019-09-30,2019,3.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,NaN,INDL,C,D,STD,USD,2019Q3,2019Q3,A


##### 各步驟篩選後的樣本量

In [16]:
# 加上cusip
cusip = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\cusip.csv")
cusip = cusip[['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic','cusip']]
# cusip['month'] = pd.to_datetime(cusip['datadate']).dt.to_period('M')
df = df.merge(cusip, on=['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic'], how='left')
df

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,lnv2s,indfmt,consol,popsrc,datafmt,curcdq,datacqtr,datafqtr,costat,cusip
0,1001,1983-03-31,1983,1.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,NaN,INDL,C,D,STD,USD,1983Q1,1983Q1,I,000165100
1,1001,1983-06-30,1983,2.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,NaN,INDL,C,D,STD,USD,1983Q2,1983Q2,I,000165100
2,1001,1983-09-30,1983,3.0,AMFD.,722.0,5812,NaN,0.185525,0.185525,...,NaN,INDL,C,D,STD,USD,1983Q3,1983Q3,I,000165100
3,1001,1983-12-31,1983,4.0,AMFD.,722.0,5812,NaN,0.031829,0.031829,...,NaN,INDL,C,D,STD,USD,1983Q4,1983Q4,I,000165100
4,1001,1984-03-31,1984,1.0,AMFD.,722.0,5812,NaN,0.031807,0.031807,...,NaN,INDL,C,D,STD,USD,1984Q1,1984Q1,I,000165100
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1659651,351491,2019-12-31,2019,4.0,IVCGF,336120.0,3711,0.039334,0.031260,0.031260,...,NaN,INDL,C,D,STD,USD,2019Q4,2019Q4,A,N47017103
1659652,351590,2019-03-31,2019,1.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,INDL,C,D,STD,USD,2019Q1,2019Q1,A,23384L101
1659653,351590,2019-06-30,2019,2.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,INDL,C,D,STD,USD,2019Q2,2019Q2,A,23384L101
1659654,351590,2019-09-30,2019,3.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,NaN,INDL,C,D,STD,USD,2019Q3,2019Q3,A,23384L101


In [17]:
# drop cusip if it is na
df = df.dropna(subset=['cusip'])

df1 = df[['gvkey','cusip']]
df1 = df1.drop_duplicates()
# 根據 cusip 欄位進行排序
df1 = df1.sort_values(by='cusip')
df1 = df1.reset_index(drop=True)
df2 = df1['cusip']
# 將 DataFrame 中的單一欄位保存到純文字文件
# df2.to_csv("output.txt", index=False)

In [18]:
df1, df2

(        gvkey      cusip
 0       23052  000021303
 1       20817  000027201
 2        1001  000165100
 3       13592  000209106
 4      147329  000255109
 ...       ...        ...
 38034   21010  Y8977Y100
 38035  166689  Y93691106
 38036  184395  Y9370Q108
 38037   20950  Y9384M101
 38038   26340  Y95308105
 
 [38039 rows x 2 columns],
 0        000021303
 1        000027201
 2        000165100
 3        000209106
 4        000255109
            ...    
 38034    Y8977Y100
 38035    Y93691106
 38036    Y9370Q108
 38037    Y9384M101
 38038    Y95308105
 Name: cusip, Length: 38039, dtype: object)

In [19]:
cusip = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\cusip convert.csv")
cusip = cusip.rename(columns={'cusip': 'crsp_cusip'})
cusip

,crsp_cusip
0,00002130
1,00002720
2,00016510
3,00020910
4,00025510
...,...
35650,Y8977Y10
35651,Y9369110
35652,Y9370Q10
35653,Y9384M10


In [20]:
df1 = df1.rename(columns={'cusip': 'compstat_cusip'})
merged_df = pd.merge(df1, cusip, left_index=True, right_index=True, how='inner')
new_df = df.merge(merged_df, on=['gvkey'], how='left')
new_df
# new_df is updated cusip df

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,consol,popsrc,datafmt,curcdq,datacqtr,datafqtr,costat,cusip,compstat_cusip,crsp_cusip
0,1001,1983-03-31,1983,1.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,C,D,STD,USD,1983Q1,1983Q1,I,000165100,000165100,00016510
1,1001,1983-06-30,1983,2.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,C,D,STD,USD,1983Q2,1983Q2,I,000165100,000165100,00016510
2,1001,1983-09-30,1983,3.0,AMFD.,722.0,5812,NaN,0.185525,0.185525,...,C,D,STD,USD,1983Q3,1983Q3,I,000165100,000165100,00016510
3,1001,1983-12-31,1983,4.0,AMFD.,722.0,5812,NaN,0.031829,0.031829,...,C,D,STD,USD,1983Q4,1983Q4,I,000165100,000165100,00016510
4,1001,1984-03-31,1984,1.0,AMFD.,722.0,5812,NaN,0.031807,0.031807,...,C,D,STD,USD,1984Q1,1984Q1,I,000165100,000165100,00016510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658432,351491,2019-12-31,2019,4.0,IVCGF,336120.0,3711,0.039334,0.031260,0.031260,...,C,D,STD,USD,2019Q4,2019Q4,A,N47017103,NaN,NaN
1658433,351590,2019-03-31,2019,1.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q1,2019Q1,A,23384L101,23384L101,25280H10
1658434,351590,2019-06-30,2019,2.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q2,2019Q2,A,23384L101,23384L101,25280H10
1658435,351590,2019-09-30,2019,3.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q3,2019Q3,A,23384L101,23384L101,25280H10


##### 把new_df存成csv，以後執行以下程式碼即可

In [2]:
# new_df.to_csv('new_df.csv', index=False)
new_df = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\new_df.csv")

In [3]:
new_df

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,consol,popsrc,datafmt,curcdq,datacqtr,datafqtr,costat,cusip,compstat_cusip,crsp_cusip
0,1001,1983-03-31,1983,1.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,C,D,STD,USD,1983Q1,1983Q1,I,000165100,000165100,00016510
1,1001,1983-06-30,1983,2.0,AMFD.,722.0,5812,NaN,NaN,NaN,...,C,D,STD,USD,1983Q2,1983Q2,I,000165100,000165100,00016510
2,1001,1983-09-30,1983,3.0,AMFD.,722.0,5812,NaN,0.185525,0.185525,...,C,D,STD,USD,1983Q3,1983Q3,I,000165100,000165100,00016510
3,1001,1983-12-31,1983,4.0,AMFD.,722.0,5812,NaN,0.031829,0.031829,...,C,D,STD,USD,1983Q4,1983Q4,I,000165100,000165100,00016510
4,1001,1984-03-31,1984,1.0,AMFD.,722.0,5812,NaN,0.031807,0.031807,...,C,D,STD,USD,1984Q1,1984Q1,I,000165100,000165100,00016510
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1658432,351491,2019-12-31,2019,4.0,IVCGF,336120.0,3711,0.039334,0.031260,0.031260,...,C,D,STD,USD,2019Q4,2019Q4,A,N47017103,NaN,NaN
1658433,351590,2019-03-31,2019,1.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q1,2019Q1,A,23384L101,23384L101,25280H10
1658434,351590,2019-06-30,2019,2.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q2,2019Q2,A,23384L101,23384L101,25280H10
1658435,351590,2019-09-30,2019,3.0,DTRUY,333924.0,3537,NaN,NaN,NaN,...,C,D,STD,USD,2019Q3,2019Q3,A,23384L101,23384L101,25280H10


In [4]:
s_e = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\share&exc code.csv")
s_e = s_e[['CUSIP','SHRCD','EXCHCD']]
# 使用 drop_duplicates 刪除重複項
s_e = s_e.drop_duplicates()

C:\Users\cindy\AppData\Local\Temp\ipykernel_16460\3470775666.py:1: DtypeWarning: Columns (4,6) have mixed types. Specify dtype option on import or set low_memory=False.
  s_e = pd.read_csv("C:\\Users\\cindy\\Desktop\\UIUC\\入學後\\Relative-Valuation\\share&exc code.csv")


In [5]:
model_df = pd.merge(new_df, s_e, left_on='crsp_cusip', right_on='CUSIP', how='left')
print(model_df)

          gvkey    datadate  fyearq  fqtr    tic     naics   sic  accrual_wr  \
0          1001  1983-03-31    1983   1.0  AMFD.     722.0  5812         NaN   
1          1001  1983-03-31    1983   1.0  AMFD.     722.0  5812         NaN   
2          1001  1983-06-30    1983   2.0  AMFD.     722.0  5812         NaN   
3          1001  1983-06-30    1983   2.0  AMFD.     722.0  5812         NaN   
4          1001  1983-09-30    1983   3.0  AMFD.     722.0  5812         NaN   
...         ...         ...     ...   ...    ...       ...   ...         ...   
2087022  351491  2019-12-31    2019   4.0  IVCGF  336120.0  3711    0.039334   
2087023  351590  2019-03-31    2019   1.0  DTRUY  333924.0  3537         NaN   
2087024  351590  2019-06-30    2019   2.0  DTRUY  333924.0  3537         NaN   
2087025  351590  2019-09-30    2019   3.0  DTRUY  333924.0  3537         NaN   
2087026  351590  2019-12-31    2019   4.0  DTRUY  333924.0  3537   -0.008326   

         aftret_eq_wr  aftret_equity_wr

In [24]:
# 1. 第一步：所有美國普通股上市公司（在指定交易所）
step1_df = model_df[(model_df['EXCHCD'].isin([1, 2, 3])) & (model_df['SHRCD'].isin([10, 11]))]
step1_firm_months = len(step1_df)
step1_unique_firms = step1_df['gvkey'].nunique()

# 2. 第二步：目標變數（m2b, v2a, v2s）為正且不缺失
step2_df = step1_df.dropna(subset=['lnm2b', 'lnv2a', 'lnv2s'])
# step2_df = step2_df[(step2_df['lnm2b'] > 0) & (step2_df['lnv2a'] > 0) & (step2_df['lnv2s'] > 0)]
step2_firm_months = len(step2_df)
step2_unique_firms = step2_df['gvkey'].nunique()

# 3. 第三步：每月所有規模變數位於第 10 百分位以上
# 提取每月的第 10 百分位值並進行篩選
step2_df['month'] = pd.to_datetime(step2_df['datadate']).dt.to_period('M')
percentiles = step2_df.groupby('month')[['atq_wr', 'saleq_wr', 'ceqq_wr']].quantile(0.1).rename(columns={
    'atq_wr': 'atq_wr_10th', 'saleq_wr': 'saleq_wr_10th', 'ceqq_wr': 'ceqq_wr_10th'
}).reset_index()

# 合併百分位數並篩選符合條件的觀察值
step3_df = step2_df.merge(percentiles, on='month', how='left')
step3_df = step3_df[(step3_df['atq_wr'] >= step3_df['atq_wr_10th']) & 
                    (step3_df['saleq_wr'] >= step3_df['saleq_wr_10th']) & 
                    (step3_df['ceqq_wr'] >= step3_df['ceqq_wr_10th'])]
step3_firm_months = len(step3_df)
step3_unique_firms = step3_df['gvkey'].nunique()

# 創建表格
summary_table = pd.DataFrame({
    '步驟': [
        '在紐約證券交易所（NYSE）、納斯達克（NASDAQ）或美國證券交易所（AMEX）上市的美國普通股',
        '目標變數（m2b, v2a, v2s）為正且不缺失',
        '每月所有規模變數（資產、賬面權益或銷售額）位於第 10 百分位以上'
    ],
    '公司-月份觀察值數量': [step1_firm_months, step2_firm_months, step3_firm_months],
    '公司數量': [step1_unique_firms, step2_unique_firms, step3_unique_firms]
})

# 顯示結果
print(summary_table)

C:\Users\cindy\AppData\Local\Temp\ipykernel_12724\3491067980.py:14: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  step2_df['month'] = pd.to_datetime(step2_df['datadate']).dt.to_period('M')


                                                  步驟  公司-月份觀察值數量   公司數量
0  在紐約證券交易所（NYSE）、納斯達克（NASDAQ）或美國證券交易所（AMEX）上市的美國普通股      809940  16073
1                          目標變數（m2b, v2a, v2s）為正且不缺失      160782   5583
2                  每月所有規模變數（資產、賬面權益或銷售額）位於第 10 百分位以上      138181   4905


In [25]:
step3_df.columns

Index(['gvkey', 'datadate', 'fyearq', 'fqtr', 'tic', 'naics', 'sic',
       'accrual_wr', 'aftret_eq_wr', 'aftret_equity_wr', 'at_turn_wr',
       'capital_ratio_wr', 'cash_debt_wr', 'cash_lt_wr', 'cash_ratio_wr',
       'cfm_wr', 'curr_debt_wr', 'curr_ratio_wr', 'de_ratio_wr',
       'debt_assets_wr', 'debt_at_wr', 'atq_wr', 'ceqq_wr', 'saleq_wr',
       'total_debt_wr', 'cash_conversion_wr', 'quick_ratio_wr', 'rd_sale_wr',
       'rect_act_wr', 'roa_wr', 'roce_wr', 'roe_wr', 'short_debt_wr',
       'totdebt_invcap_wr', 'rect_turn_wr', 'debt_capital_wr',
       'debt_ebitda_wr', 'debt_invcap_wr', 'dltt_be_wr', 'dpr_wr', 'efftax_wr',
       'equity_invcap_wr', 'fcf_ocf_wr', 'gpm_wr', 'gprof_wr', 'int_debt_wr',
       'int_totdebt_wr', 'intcov_ratio_wr', 'intcov_wr', 'invt_act_wr',
       'lt_debt_wr', 'lt_ppent_wr', 'npm_wr', 'ocf_lct_wr', 'opmad_wr',
       'opmbd_wr', 'pretret_earnat_wr', 'pretret_noa_wr', 'profit_lct_wr',
       'ptpm_wr', 'invturn_wr', 'pay_turn_wr', 'm2b', 'v2a', 

##### add in fama-french 49 industries categories

In [26]:
sic_data = [
    ["Agric", "0100-0799"],
    ["Agric", "2048-2048"],
    ["Food", "2000-2046"],
    ["Food", "2050-2063"],
    ["Food", "2070-2079"],
    ["Food", "2090-2095"],
    ["Food", "2098-2099"],
    ["Soda", "2064-2068"],
    ["Soda", "2086-2087"],
    ["Soda", "2096-2097"],
    ["Beer", "2080-2085"],
    ["Smoke", "2100-2199"],
    ["Toys", "0900-0999"],
    ["Toys", "3650-3652"],
    ["Toys", "3732-3732"],
    ["Toys", "3930-3949"],
    ["Fun", "7800-7841"],
    ["Fun", "7900-7999"],
    ["Books", "2700-2749"],
    ["Books", "2770-2799"],
    ["Hshld", "2047-2047"],
    ["Hshld", "2391-2392"],
    ["Hshld", "2510-2519"],
    ["Hshld", "2590-2599"],
    ["Hshld", "2840-2844"],
    ["Hshld", "3160-3199"],
    ["Hshld", "3229-3231"],
    ["Hshld", "3260-3260"],
    ["Hshld", "3262-3263"],
    ["Hshld", "3269-3269"],
    ["Hshld", "3630-3639"],
    ["Hshld", "3750-3751"],
    ["Hshld", "3800-3800"],
    ["Hshld", "3860-3879"],
    ["Hshld", "3910-3919"],
    ["Hshld", "3960-3961"],
    ["Hshld", "3991-3991"],
    ["Hshld", "3995-3995"],
    ["Clths", "2300-2390"],
    ["Clths", "3020-3021"],
    ["Clths", "3100-3111"],
    ["Clths", "3130-3159"],
    ["Clths", "3965-3965"],
    ["Hlth", "8000-8099"],
    ["MedEq", "3693-3693"],
    ["MedEq", "3840-3851"],
    ["Drugs", "2830-2836"],
    ["Chems", "2800-2829"],
    ["Chems", "2850-2899"],
    ["Rubbr", "3000-3000"],
    ["Rubbr", "3050-3099"],
    ["Txtls", "2200-2295"],
    ["Txtls", "2297-2299"],
    ["Txtls", "2393-2395"],
    ["Txtls", "2397-2399"],
    ["BIdMt", "0800-0899"],
    ["BIdMt", "2400-2439"],
    ["BIdMt", "2450-2459"],
    ["BIdMt", "2490-2499"],
    ["BIdMt", "2950-2952"],
    ["BIdMt", "3200-3219"],
    ["BIdMt", "3240-3259"],
    ["BIdMt", "3261-3261"],
    ["BIdMt", "3264-3264"],
    ["BIdMt", "3270-3299"],
    ["BIdMt", "3420-3442"],
    ["BIdMt", "3446-3452"],
    ["BIdMt", "3490-3499"],
    ["BIdMt", "3996-3996"],
    ["Construction", "1500-1549"],
    ["Construction", "1600-1699"],
    ["Construction", "1700-1799"],
    ["Steel", "3300-3369"],
    ["Steel", "3390-3399"],
    ["FabPr", "3400-3400"],
    ["FabPr", "3443-3444"],
    ["FabPr", "3460-3479"],
    ["Mach", "3510-3536"],
    ["Mach", "3540-3569"],
    ["Mach", "3580-3599"],
    ["ElcEq", "3600-3621"],
    ["ElcEq", "3623-3629"],
    ["ElcEq", "3640-3646"],
    ["ElcEq", "3648-3649"],
    ["ElcEq", "3660-3660"],
    ["ElcEq", "3691-3692"],
    ["ElcEq", "3699-3699"],
    ["Misc", "3900-3900"],
    ["Misc", "3990-3990"],
    ["Misc", "3999-3999"],
    ["Misc", "9900-9999"],
    ["Autos", "2296-2296"],
    ["Autos", "2396-2396"],
    ["Autos", "3010-3011"],
    ["Autos", "3537-3537"],
    ["Autos", "3647-3647"],
    ["Autos", "3694-3694"],
    ["Autos", "3700-3716"],
    ["Autos", "3790-3792"],
    ["Autos", "3799-3799"],
    ["Aero", "3720-3729"],
    ["Ships", "3730-3731"],
    ["Ships", "3740-3743"],
    ["Guns", "3480-3489"],
    ["Guns", "3760-3769"],
    ["Guns", "3795-3795"],
    ["Gold", "1040-1049"],
    ["Mines", "1000-1039"],
    ["Mines", "1060-1099"],
    ["Mines", "1400-1499"],
    ["Coal", "1200-1299"],
    ["Enrgy", "1310-1389"],
    ["Enrgy", "2900-2911"],
    ["Enrgy", "2990-2999"],
    ["Uti", "4900-4999"],
    ["TeIcm", "4800-4899"],
    ["PerSv", "7020-7021"],
    ["PerSv", "7030-7039"],
    ["PerSv", "7200-7212"],
    ["PerSv", "7215-7299"],
    ["PerSv", "7395-7395"],
    ["PerSv", "7500-7500"],
    ["PerSv", "7520-7549"],
    ["PerSv", "7600-7699"],
    ["PerSv", "8100-8199"],
    ["PerSv", "8200-8299"],
    ["PerSv", "8300-8399"],
    ["PerSv", "8400-8499"],
    ["PerSv", "8600-8699"],
    ["PerSv", "8800-8899"],
    ["BusSv", "2750-2759"],
    ["BusSv", "3993-3993"],
    ["BusSv", "7300-7372"],
    ["BusSv", "7374-7394"],
    ["BusSv", "7397-7397"],
    ["BusSv", "7399-7399"],
    ["BusSv", "7510-7519"],
    ["BusSv", "8700-8748"],
    ["BusSv", "8900-8999"],
    ["Comps", "3570-3579"],
    ["Comps", "3680-3689"],
    ["Comps", "3695-3695"],
    ["Comps", "7373-7373"],
    ["Chips", "3622-3622"],
    ["Chips", "3661-3679"],
    ["Chips", "3810-3810"],
    ["Chips", "3812-3812"],
    ["LabEq", "3811-3811"],
    ["LabEq", "3820-3830"],
    ["Paper", "2520-2549"],
    ["Paper", "2600-2639"],
    ["Paper", "2670-2699"],
    ["Paper", "2760-2761"],
    ["Paper", "3950-3955"],
    ["Boxes", "2440-2449"],
    ["Boxes", "2640-2659"],
    ["Boxes", "3210-3221"],
    ["Boxes", "3410-3412"],
    ["Trans", "4000-4099"],
    ["Trans", "4100-4199"],
    ["Trans", "4200-4299"],
    ["Trans", "4400-4499"],
    ["Trans", "4500-4599"],
    ["Trans", "4600-4699"],
    ["Trans", "4700-4799"],
    ["Whlsl", "5000-5099"],
    ["Whlsl", "5100-5199"],
    ["Rtail", "5200-5299"],
    ["Rtail", "5300-5399"],
    ["Rtail", "5400-5499"],
    ["Rtail", "5500-5599"],
    ["Rtail", "5600-5699"],
    ["Rtail", "5700-5736"],
    ["Rtail", "5900-5999"],
    ["Meals", "5800-5813"],
    ["Meals", "5890-5890"],
    ["Meals", "7000-7019"],
    ["Meals", "7040-7049"],
    ["Meals", "7213-7213"],
    ["Banks", "6000-6099"],
    ["Banks", "6100-6199"],
    ["Insur", "6300-6399"],
    ["Insur", "6400-6411"],
    ["RlBst", "6500-6553"],
    ["Fin", "6200-6299"],
    ["Fin", "6700-6799"]
]
# Create a DataFrame from the data
sic_df = pd.DataFrame(sic_data, columns=["Division", "Range of SIC Codes"])

# Split the 'Range of SIC Codes' into two new columns 'Lower Bound' and 'Upper Bound'
sic_df[['Lower Bound', 'Upper Bound']] = sic_df['Range of SIC Codes'].str.split('-', expand=True)

# Display the resulting DataFrame
print(sic_df[['Lower Bound', 'Upper Bound', 'Division']])

    Lower Bound Upper Bound Division
0          0100        0799    Agric
1          2048        2048    Agric
2          2000        2046     Food
3          2050        2063     Food
4          2070        2079     Food
..          ...         ...      ...
181        6300        6399    Insur
182        6400        6411    Insur
183        6500        6553    RlBst
184        6200        6299      Fin
185        6700        6799      Fin

[186 rows x 3 columns]


In [27]:
sic_df['Lower Bound'] = pd.to_numeric(sic_df['Lower Bound'])
sic_df['Upper Bound'] = pd.to_numeric(sic_df['Upper Bound'])

# Function to lookup the division
def find_division(sic_code):
    division = sic_df[(sic_df['Lower Bound'] <= sic_code) & (sic_df['Upper Bound'] >= sic_code)]['Division']
    return division.iloc[0] if not division.empty else 'Unknown'

# Apply the function to the 'sic' column
# 使用 LabelEncoder 編碼產業分類
label_encoder = LabelEncoder()
step3_df['industry'] = step3_df['sic'].apply(find_division)
step3_df['famafrench_wr'] = label_encoder.fit_transform(step3_df['industry'])
step3_df

,gvkey,datadate,fyearq,fqtr,tic,naics,sic,accrual_wr,aftret_eq_wr,aftret_equity_wr,...,crsp_cusip,CUSIP,SHRCD,EXCHCD,month,atq_wr_10th,saleq_wr_10th,ceqq_wr_10th,industry,famafrench_wr
0,1004,2006-08-31,2006,1.0,AIR,423860.0,5080,-0.014385,0.028141,0.028141,...,00036110,00036110,11.0,2.0,2006-08,6.1938,0.0,2.7176,Whlsl,48
1,1004,2006-08-31,2006,1.0,AIR,423860.0,5080,-0.014385,0.028141,0.028141,...,00036110,00036110,11.0,1.0,2006-08,6.1938,0.0,2.7176,Whlsl,48
2,1004,2006-11-30,2006,2.0,AIR,423860.0,5080,-0.009567,0.030985,0.030985,...,00036110,00036110,11.0,2.0,2006-11,5.2480,0.0,2.5520,Whlsl,48
3,1004,2006-11-30,2006,2.0,AIR,423860.0,5080,-0.009567,0.030985,0.030985,...,00036110,00036110,11.0,1.0,2006-11,5.2480,0.0,2.5520,Whlsl,48
4,1004,2007-02-28,2006,3.0,AIR,423860.0,5080,-0.014462,0.033040,0.033040,...,00036110,00036110,11.0,2.0,2007-02,4.8644,0.0,3.6776,Whlsl,48
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
160777,315318,2018-12-31,2018,4.0,ESI,325998.0,2890,0.000457,-0.006163,-0.006163,...,29647510,29647510,11.0,3.0,2018-12,15.6460,0.0,7.7352,Chems,9
160778,315318,2019-03-31,2019,1.0,ESI,325998.0,2890,-0.031451,-0.001817,-0.001817,...,29647510,29647510,11.0,3.0,2019-03,15.6040,0.0,7.3691,Chems,9
160779,315318,2019-06-30,2019,2.0,ESI,325998.0,2890,-0.032588,0.006584,0.006584,...,29647510,29647510,11.0,3.0,2019-06,14.6960,0.0,7.5042,Chems,9
160780,315318,2019-09-30,2019,3.0,ESI,325998.0,2890,-0.013105,-0.002827,-0.002827,...,29647510,29647510,11.0,3.0,2019-09,14.3720,0.0,8.0535,Chems,9


##### 建立模型

In [ ]:
import numpy as np
import lightgbm as lgb
from numpy import set_printoptions
from sklearn.datasets import make_friedman1
from sklearn.datasets import make_regression
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [29]:
# 設定了 NumPy 的印出格式，precision=4 代表小數點後保留 4 位數
# linewidth=100 表示每行顯示的最大寬度為 100
set_printoptions(precision=4)
np.set_printoptions(linewidth=100)

In [ ]:
# test settings - 觀測數量、學習率、樹的數量以及每棵樹的葉節點數量
LEARNING_RATE = 0.1
TREES = 12
LEAVES = 5

In [31]:
# 函數 tree_prediction 用來從梯度提升模型（GBM）中提取單棵樹的預測值
# 根據不同的樹迭代次數，它會返回該樹的預測值
# 如果是第一棵樹（tree_iteration == 0），它返回訓練樣本的平均值；
# 如果是第二棵樹（tree_iteration == 1），它會調整預測值；
# 否則，它會返回指定樹的預測結果
def tree_prediction(model, data, train_sample_average, tree_iteration, learning_rate):
    '''
    extracts individual tree predictions from GBM model
    '''
    if tree_iteration == 0:
        tree_prediction = train_sample_average
    elif tree_iteration == 1:
        gbm1_prediction = model.predict(data, start_iteration=0, num_iteration=1)    
        tree_prediction = (gbm1_prediction - train_sample_average) / learning_rate
    else:
        tree_prediction = model.predict(data, start_iteration=tree_iteration-1, num_iteration=1) / learning_rate
    return tree_prediction

In [32]:
# 葉節點成員矩陣生成函數
def getD(tree):
    '''
    utility function to create leaf membership matrix D from leaf membership vector tree
    element d_f,c in D takes value of 1 IFF observations f and c are allocated to the same leaf
    that is, tree[f] == tree[c], otherwise 0
    '''
    # 使用 NumPy 來替代雙重迴圈
    tree = np.array(tree)  # 確保 tree 是一個 NumPy 陣列
    D = (tree[:, None] == tree[None, :]).astype(int)
    return D
# getD 會從樹的葉節點生成一個成員矩陣 D。D[f, c] 
# 只有在觀測值 f 和 c 分配到同一個葉節點時才為 1，否則為 0
# tree[:, None] 和 tree[None, :]： 這種方式創建了樹向量的兩個不同視圖，一個是列向量，另一個是行向量。這樣可以進行廣播操作，快速比較每個元素，生成布林矩陣。
# .astype(int)： 將布林矩陣轉換為整數矩陣，以符合原始函數中的邏輯。

In [38]:
# 選擇特徵欄位和目標欄位
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_column = ['lnm2b']
# target_column = ['lnm2b','lnv2a','lnv2s']
# 生成特徵矩陣 X 和目標變數 y
X = step3_df[feature_columns].to_numpy()
y = step3_df[target_column].to_numpy()

# 確認資料形狀
print("X shape:", X.shape)
print("y shape:", y.shape)

X shape: (138181, 56)
y shape: (138181, 1)


In [39]:
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 2,
    "learning_rate": LEARNING_RATE,    
}
# boosting_type: 設定提升樹的類型，這裡使用的是 gbdt（Gradient Boosting Decision Tree，梯度提升決策樹）。
# objective: 模型的目標函數，這裡設定為 regression，表示進行回歸分析。
# metric: 評估指標，這裡使用 rmse（均方根誤差）來評估模型的預測誤差。
# num_leaves: 決策樹中葉節點的數量，這裡設定為 LEAVES（之前設定為 5），葉節點越多，模型越複雜。
# verbose: 設定輸出的詳細程度，1 表示顯示基本的運行資訊。
# min_data: 每個葉節點的最小資料量，這裡設定為 2，避免過度擬合。
# learning_rate: 學習率，這裡使用 LEARNING_RATE（之前設定為 0.1），學習率決定了模型每次更新的步伐大小。

In [40]:
# 訓練 LightGBM 模型
data = lgb.Dataset(X, label=y)
model = lgb.train(params, data, num_boost_round=TREES-1)
y_hat = model.predict(X)
y_hat.shape  # 輸出預測結果的形狀

# 獲取每個樣本在每棵樹中的葉節點編號
instance_leaf_membership = model.predict(X, pred_leaf=True)

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.015820 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14074
[LightGBM] [Info] Number of data points in the train set: 138181, number of used features: 56
[LightGBM] [Info] Start training from score 0.257248


c:\Users\cindy\AppData\Local\Programs\Python\Python312\Lib\site-packages\lightgbm\basic.py:357: UserWarning: Converting column-vector to 1d array
  _log_warning("Converting column-vector to 1d array")


In [41]:
# 生成並顯示每棵樹的結構圖
for index in range(1, TREES):
    graph = lgb.create_tree_digraph(booster=model, tree_index=index-1,
                    show_info=['internal_count', 'leaf_count', 
                               'leaf_weight', 'data_percentage', 'internal_value'],
                    name="Tree"+str(index))
    graph.render(view=True)

In [ ]:
# 提取 GBM 模型中的樹預測
OBSERVATIONS = X.shape[0]  # 將 OBSERVATIONS 設定為資料集的行數
train_average = y.mean()
tree = []
tp = np.full(OBSERVATIONS, train_average)
tree.append(tp)
for index in range(1, TREES):
    tp = tree_prediction(model=model, data=X, train_sample_average=train_average, tree_iteration=index, learning_rate=LEARNING_RATE)
    tree.append(tp)
# train_average：目標變數 y 的平均值，用於初始化第一棵樹的預測值。
# tree_prediction：提取指定樹的預測值，並將結果添加到 tree 列表中。

In [45]:
ensemble = tree[0]
for index in range(1, TREES):
    ensemble += LEARNING_RATE * tree[index]

# 確認手動構建的預測值與模型的預測值一致
diff = ensemble - y_hat
assert(np.allclose(diff, np.zeros((1, OBSERVATIONS)), atol=1e-6))
# ensemble：用來累加每棵樹的預測值，最後得到整體預測。
# np.allclose：用來檢查兩個數組是否幾乎相等，確認手動預測與模型預測一致。

##### chatgpt

In [66]:
# 設定樹的數量範圍
tree_options = [10, 20, 50, 100]
best_tree_count = 0
best_rmse = float("inf")

# 定義參數
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": 31,
    "min_data_in_leaf": 20,
    "learning_rate": 0.1,
}

# 提取特徵和目標變數
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
X = step3_df[feature_columns].to_numpy()
y = step3_df['lnm2b'].to_numpy()  # 假設用一個目標變數進行測試

# 分割訓練集和驗證集
X_train, X_val, y_train, y_val = train_test_split(X, y, test_size=0.2, random_state=42)

# 使用不同的樹數進行訓練並記錄 RMSE
for num_trees in tree_options:
    train_data = lgb.Dataset(X_train, label=y_train)
    model = lgb.train(params, train_data, num_boost_round=num_trees)
    
    # 驗證集預測
    y_val_pred = model.predict(X_val)
    rmse = np.sqrt(np.mean((y_val_pred - y_val) ** 2))

    print(f"Number of Trees: {num_trees}, RMSE: {rmse}")

    # 記錄最佳樹數
    if rmse < best_rmse:
        best_rmse = rmse
        best_tree_count = num_trees

print(f"Optimal Number of Trees: {best_tree_count}, Best RMSE: {best_rmse}")

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.012783 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14074
[LightGBM] [Info] Number of data points in the train set: 110544, number of used features: 56
[LightGBM] [Info] Start training from score 0.256908
Number of Trees: 10, RMSE: 0.3126618807696022
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.016949 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14074
[LightGBM] [Info] Number of data points in the train set: 110544, number of used features: 56
[LightGBM] [Info] Start training from score 0.256908
Number of Trees: 20, RMSE: 0.2871057814536065
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.027005 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 14074
[LightGBM] [Info] Numbe

In [62]:
LEARNING_RATE = 0.1
LEAVES = 31
TREES = 100  # 固定迭代次數，設定為100
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 20,
    "learning_rate": LEARNING_RATE,
}

# 假設 step3_df 是你的完整資料集
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_columns = ['lnm2b']
# 添加月份列
step3_df['month'] = pd.to_datetime(step3_df['datadate']).dt.to_period('M')

# 儲存樣本外預測結果
oos_predictions = {target: [] for target in target_columns}

# 每月處理數據
for month, monthly_data in step3_df.groupby('month'):
    print(f"Processing month: {month}")

    # 提取特徵矩陣 X 和多目標變數 y
    X = monthly_data[feature_columns].to_numpy()
    y_all = monthly_data[target_columns].to_numpy()

    # 將資料分為 60% 訓練集、20% 驗證集、20% 測試集
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_all, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # 隨機分配到 5 個區塊
    blocks = np.array_split(np.random.permutation(len(X_train)), 5)

    # 對於每個目標變數，進行五折交叉驗證
    for target_idx, target in enumerate(target_columns):
        print(f"Training models for target: {target}")

        # 訓練資料集
        train_data = lgb.Dataset(X_train, label=y_train[:, target_idx])

        # 使用固定迭代次數訓練模型
        model = lgb.train(
            params,
            train_data,
            num_boost_round=TREES
        )

        # 使用固定迭代次數進行預測
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        # 儲存測試結果
        oos_predictions[target].append({
            'month': month,
            'y_val_pred': y_val_pred,
            'y_test_pred': y_test_pred,
            'y_test_actual': y_test[:, target_idx]
        })

# 最終的樣本外預測結果儲存到 DataFrame 中便於分析
oos_df = pd.DataFrame({
    'month': [entry['month'] for target in target_columns for entry in oos_predictions[target]],
    'target': [target for target in target_columns for _ in oos_predictions[target]],
    'y_val_pred': [entry['y_val_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_pred': [entry['y_test_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_actual': [entry['y_test_actual'] for target in target_columns for entry in oos_predictions[target]]
})

# 查看結果
print(oos_df.head())

Processing month: 2005-09
Training models for target: lnm2b
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000144 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1316
[LightGBM] [Info] Number of data points in the train set: 78, number of used features: 55
[LightGBM] [Info] Start training from score 0.392143
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

##### lnv2s

In [63]:
LEARNING_RATE = 0.05
LEAVES = 31
TREES = 100  # 固定迭代次數，設定為100
params = {
    "boosting_type": "gbdt",
    "objective": "regression",
    "metric": "rmse",
    "num_leaves": LEAVES,
    "verbose": 1,
    "min_data": 20,
    "learning_rate": LEARNING_RATE,
}

# 假設 step3_df 是你的完整資料集
feature_columns = [col for col in step3_df.columns if col.endswith('wr')]
target_columns = ['lnv2s']
# 添加月份列
step3_df['month'] = pd.to_datetime(step3_df['datadate']).dt.to_period('M')

# 儲存樣本外預測結果
oos_predictions = {target: [] for target in target_columns}

# 每月處理數據
for month, monthly_data in step3_df.groupby('month'):
    print(f"Processing month: {month}")

    # 提取特徵矩陣 X 和多目標變數 y
    X = monthly_data[feature_columns].to_numpy()
    y_all = monthly_data[target_columns].to_numpy()

    # 將資料分為 60% 訓練集、20% 驗證集、20% 測試集
    X_train_val, X_test, y_train_val, y_test = train_test_split(X, y_all, test_size=0.2, random_state=42)
    X_train, X_val, y_train, y_val = train_test_split(X_train_val, y_train_val, test_size=0.25, random_state=42)

    # 隨機分配到 5 個區塊
    blocks = np.array_split(np.random.permutation(len(X_train)), 5)

    # 對於每個目標變數，進行五折交叉驗證
    for target_idx, target in enumerate(target_columns):
        print(f"Training models for target: {target}")

        # 訓練資料集
        train_data = lgb.Dataset(X_train, label=y_train[:, target_idx])

        # 使用固定迭代次數訓練模型
        model = lgb.train(
            params,
            train_data,
            num_boost_round=TREES
        )

        # 使用固定迭代次數進行預測
        y_val_pred = model.predict(X_val)
        y_test_pred = model.predict(X_test)

        # 儲存測試結果
        oos_predictions[target].append({
            'month': month,
            'y_val_pred': y_val_pred,
            'y_test_pred': y_test_pred,
            'y_test_actual': y_test[:, target_idx]
        })

# 最終的樣本外預測結果儲存到 DataFrame 中便於分析
oos_df = pd.DataFrame({
    'month': [entry['month'] for target in target_columns for entry in oos_predictions[target]],
    'target': [target for target in target_columns for _ in oos_predictions[target]],
    'y_val_pred': [entry['y_val_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_pred': [entry['y_test_pred'] for target in target_columns for entry in oos_predictions[target]],
    'y_test_actual': [entry['y_test_actual'] for target in target_columns for entry in oos_predictions[target]]
})

# 查看結果
print(oos_df.head())

Processing month: 2005-09
Training models for target: lnv2s
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.000191 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 1316
[LightGBM] [Info] Number of data points in the train set: 78, number of used features: 55
[LightGBM] [Info] Start training from score 0.795327
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[

------

In [46]:
# 生成葉節點成員矩陣
def getD(tree):
    '''
    用來生成葉節點成員矩陣 D
    如果樣本 f 和 c 位於同一葉節點，則 D[f, c] = 1；否則為 0
    '''
    tree = np.array(tree)  # 確保 tree 是 NumPy 陣列
    D = (tree[:, None] == tree[None, :]).astype(int)
    return D

##### 根據 Geertsema & Lu (2022) 的公式進行計算，計算過程涉及多棵樹的預測和觀測之間的權重更新，目的是模擬一種相對估值模型。

In [47]:
# ones：建立全為 1 的矩陣，方便矩陣操作。
# I：單位矩陣，用於保持數值計算的穩定性。
# P, p, diff, G, g, D, W：儲存每棵樹中各步驟的計算結果。
N = OBSERVATIONS  # 樣本數量
M = TREES         # 樹的數量
_lambda = LEARNING_RATE  # 學習率

ones = np.ones((N, N))  # 全部為 1 的矩陣
I = np.identity(N)      # 單位矩陣 (N x N)

# 初始化各變數的列表
P, p, diff, G, g, D, W = [[None] * M for _ in range(7)]

MemoryError: Unable to allocate 142. GiB for an array with shape (138181, 138181) and data type float64

In [ ]:
# 迭代第一棵樹的初始化 (i=0)
P[0] = (1 / N) * ones  # 初始化 P[0] 矩陣 (方程 11)
v = y.reshape(N, 1).copy()  # 目標值 y 的向量化

p[0] = v.T @ P[0]  # 初始條件下的 p[0] 計算 (方程 12)
t = np.reshape(tree[0], (1, N))
diff[0] = t - p[0]  # 確認 p[0] 與 tree[0] 的一致性
assert np.allclose(np.mean(tree[0]), np.mean(p[0]), atol=1e-6) # 驗證差異接近零

G[0] = P[0]  # 初始 G[0] 設為 P[0] (方程 13)
g[0] = v.T @ G[0]  # 初始 g[0] 計算 (方程 14)
D[0] = np.ones((N, N))  # 第一棵樹的 D 矩陣，所有樣本在同一葉節點
# P[0]：初始條件下的權重矩陣。
# p[0]：根據初始條件的預測值。
# diff[0]：檢查 p[0] 是否與第一棵樹的預測一致。
# D[0]：所有觀測分配到同一葉節點。

In [ ]:
# 迭代樹 (i ≥ 1)
for i in range(1, M):
    D[i] = getD(tree[i])  # 使用 getD 函數生成 D[i] 矩陣
    W[i] = D[i] / (ones @ D[i])  # W[i] 矩陣計算 (方程 15)

    P[i] = _lambda * (W[i] @ (I - G[i-1]))  # P[i] 計算 (方程 16)
    p[i] = v.T @ P[i].T @ W[i]  # p[i] 計算 (方程 17)

    t = np.reshape(tree[i], (1, N))
    diff[i] = (t * _lambda) - (p[i])  # 檢查差異
    assert(np.allclose(diff[i], np.zeros((1, N)), atol=1e-5))  # 確認差異接近零

    G[i] = G[i-1] + P[i]  # G[i] 累積計算 (方程 18)
    g[i] = v.T @ G[i].T  # g[i] 計算 (方程 19)
# D[i]：第 i 棵樹的葉節點成員矩陣，使用 getD 函數生成。
# W[i]：權重矩陣（方程 15）。
# P[i]：當前迭代的權重更新（方程 16）。
# p[i]：第 i 棵樹的預測值（方程 17）。
# diff[i]：檢查 p[i] 是否與樹的預測一致。
# G[i] 和 g[i]：累積矩陣和對應的向量更新（方程 18 和 19）。

##### 通過計算權重來實現類似 GBM 的預測，並將預測結果與實際的 GBM 預測進行對比，以檢查差異

In [ ]:
# 初始化 L 矩陣
# L 矩陣儲存每棵樹的葉節點分配信息
L = D.copy()

# 在第一棵樹中，所有觀測都分配到同一葉節點
L[0] = np.ones((N, N))  # 第一棵樹的所有值均為 1

In [ ]:
# 利用計算出的權重進行 GBM 預測
K = np.zeros((N, N))
for i in range(0, M):
    # 方程 20：累加每棵樹的預測權重
    K = K + P[i].T @ (L[i] / (ones @ L[i]))

In [ ]:
# 方程 21：計算最終預測值向量 k
k = v.T @ K

# 檢查 k 與 y_hat 的差異
diff_pred = k - y_hat
print("===CHECK===: difference in GBM predicted ", np.round(diff_pred,6))

# 檢查差異是否在允許範圍內
assert(np.allclose(diff_pred, np.zeros((1, N)), atol=1e-5))
print("checks complete")